In [16]:
##This code is for determining my initial list of stocks. I have a twitter account that 
##follows 150 penny stock focused day/swing traders each with at least 1000 followers. 

#I have generally found that on Sundays everyone is talking about what stocks they like for
#the week. This code collects all of the tweets from that timeline on Sunday and exports 
##them to a .json file that you need to name
#(fine tune how far back in time it goes by setting the number of pages, 3 is generally
#around the right amount when i run this at 11 pm EST). 
#Then it reads the .json file (need to set file name for the read) and tallies up how 
#many times each stock got mentioned on the timeline and exports a list of 
#stock name and mentions. A lot of this initial chunk i stitched together from various places
#on the internet and did not write
##In the next step of this code, it takes the list of stocks and using the WeBull API checks
# to see if they are available for trading, and also filters them to make sure they aren't something
##super legit and non-volatile.
##it then automatically calculates several machine learning model features and exports to
# a spreadsheet 
##I am pretty sure you need to be logged into webull in order to access the webull API data. 
from tweepy import OAuthHandler
import operator 
import json
import nltk
import tweepy
from tweepy import Cursor
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string
import itertools
import re
##these are for accessing the twitter account, there is rate limiting on this i forget 
##what the rules are exactly but look them up if you are gonna try and get more than ~1,000
#tweets a couple times in a row I'd say (will get kicked if you break there rules too much)
consumer_key = "XXXXXXXXXXXXXXXXXX"
consumer_secret = "XXXXXXXXXXXXXXXX"
access_token = "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"
access_secret = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
#dont want to put the keys on the internet
# Creating the authentication object
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
from webull import webull
import xlsxwriter
api = tweepy.API(auth)
wb = webull()


In [17]:
if __name__ == '__main__':
    ####################################set export file name for json here
    with open ('home_timelineTESTTEST.json1', 'w') as f:
        ##################################################################fine tune how many 
        ###tweets you are reading here 
        for page in tweepy.Cursor(api.home_timeline, count = 200).pages(3):
        ##################################################################
            for status in page:
                f.write(json.dumps(status._json)+ "\n")

In [18]:

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\$+[\w_]+[\w\'_\-]*[\w_]+)", # stonks
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
  
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str, re.VERBOSE | re.IGNORECASE)
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via', 'I', 'RT', 'DD', 'SPY', 'PR', 'A', 'THIS']
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
######file you want to read 
fname = 'home_timelineTESTTEST.json1'
#######################
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_all = [term for term in preprocess(tweet['text'])if len(term)<=6 and term not in stop and term.startswith('$') and term.isupper()]
        # Update the counter
        count_all.update(terms_all)
        a = count_all.most_common(300)
    print(count_all.most_common(300))

[('$USWS', 21), ('$NAKD', 21), ('$CCIV', 20), ('$TTOO', 13), ('$TMBR', 10), ('$SNDL', 9), ('$ZOM', 9), ('$CLOV', 8), ('$JUPW', 8), ('$LMFA', 7), ('$ACST', 7), ('$ISR', 6), ('$BNGO', 6), ('$NIO', 6), ('$AVGR', 6), ('$XSPA', 6), ('$TSLA', 4), ('$GOVX', 4), ('$THMO', 4), ('$XPEL', 4), ('$ZM', 3), ('$SIFY', 3), ('$TRXC', 3), ('$MESO', 3), ('$GEVO', 3), ('$NVDA', 3), ('$CTRM', 3), ('$LEAP', 3), ('$AIKI', 2), ('$DFFN', 2), ('$NSPR', 2), ('$RIOT', 2), ('$CBMJ', 2), ('$KMPH', 2), ('$NBY', 2), ('$LLY', 2), ('$BTWN', 2), ('$CRBP', 2), ('$EXPC', 2), ('$SRPT', 2), ('$TLRY', 2), ('$SIGL', 2), ('$CIDM', 2), ('$RIDE', 2), ('$SLGG', 2), ('$NTEC', 2), ('$ACB', 2), ('$IDEX', 2), ('$OCGN', 2), ('$FB', 2), ('$TAK', 2), ('$GHSI', 2), ('$BRQS', 2), ('$CHNR', 2), ('$ELTK', 2), ('$TGC', 2), ('$BW', 1), ('$AEZS', 1), ('$NPA', 1), ('$GRNQ', 1), ('$UVXY', 1), ('$TSNP', 1), ('$PYPL', 1), ('$SQ', 1), ('$MGNI', 1), ('$MARA', 1), ('$NMRD', 1), ('$PENN', 1), ('$STTH', 1), ('$UPIN', 1), ('$PGAS', 1), ('$INUV', 1), ('$

In [20]:
b = list(itertools.chain(*a))
print(b)
names = b[::2]
mentions = b[1::2]
print(names)
print(mentions)
all_names = [] 
not_listed = [] 
NA_stocks = [] ##stocks not available for trading
too_legit = [] ##stuff that is too legit
just_right = [] ##stuff that is just right 
just_right_mentions = [] 
for i in range(len(names)):
    name = names[i]
    final_name = name.strip('$')
    all_names.append(final_name)
    #print(final_name)
    try:
        b = wb.get_quote(final_name)
        if b['status'] == 'B':
            NA_stocks.append(final_name)
        elif b['status'] == 'A':
            if float(b['close'])>= 40.0 or float(b['fiftyTwoWkLow'])>= 10.0:
                too_legit.append(final_name)
            else:
                just_right.append(final_name)
                just_right_mentions.append(mentions[i])
    except ValueError:
        not_listed.append(final_name)
        continue 
    except KeyError:
        not_listed.append(final_name)
        continue 
    #b = wb.get_bars(final_name)
print("Just Right",just_right, "too legit", too_legit, "not available", NA_stocks)







['$USWS', 21, '$NAKD', 21, '$CCIV', 20, '$TTOO', 13, '$TMBR', 10, '$SNDL', 9, '$ZOM', 9, '$CLOV', 8, '$JUPW', 8, '$LMFA', 7, '$ACST', 7, '$ISR', 6, '$BNGO', 6, '$NIO', 6, '$AVGR', 6, '$XSPA', 6, '$TSLA', 4, '$GOVX', 4, '$THMO', 4, '$XPEL', 4, '$ZM', 3, '$SIFY', 3, '$TRXC', 3, '$MESO', 3, '$GEVO', 3, '$NVDA', 3, '$CTRM', 3, '$LEAP', 3, '$AIKI', 2, '$DFFN', 2, '$NSPR', 2, '$RIOT', 2, '$CBMJ', 2, '$KMPH', 2, '$NBY', 2, '$LLY', 2, '$BTWN', 2, '$CRBP', 2, '$EXPC', 2, '$SRPT', 2, '$TLRY', 2, '$SIGL', 2, '$CIDM', 2, '$RIDE', 2, '$SLGG', 2, '$NTEC', 2, '$ACB', 2, '$IDEX', 2, '$OCGN', 2, '$FB', 2, '$TAK', 2, '$GHSI', 2, '$BRQS', 2, '$CHNR', 2, '$ELTK', 2, '$TGC', 2, '$BW', 1, '$AEZS', 1, '$NPA', 1, '$GRNQ', 1, '$UVXY', 1, '$TSNP', 1, '$PYPL', 1, '$SQ', 1, '$MGNI', 1, '$MARA', 1, '$NMRD', 1, '$PENN', 1, '$STTH', 1, '$UPIN', 1, '$PGAS', 1, '$INUV', 1, '$MSOS', 1, '$KBWB', 1, '$EXAS', 1, '$IPOC', 1, '$GMEV', 1, '$GSL', 1, '$NOVN', 1, '$ATHE', 1, '$BORR', 1, '$UPWK', 1, '$GME', 1, '$FCEL', 1, '$NGA

In [22]:

twitter_rank = [] 
friday_close = [] 
friday_volume = [] 
three_month_volume = [] 
fifty_two_week_low = [] 
fifty_two_week_high = [] 

for i in range(len(just_right)):
    for x in range(len(all_names)):
        if just_right[i] == all_names[x]:
            x = x+1
            twitter_rank.append(x)
    quote = wb.get_quote(just_right[i])
    friday_close.append(float(quote['close']))
    friday_volume.append(float(quote['volume']))
    three_month_volume.append(float(quote['avgVol3M']))
    fifty_two_week_low.append(float(quote['fiftyTwoWkLow']))
    fifty_two_week_high.append(float(quote['fiftyTwoWkHigh']))
print(just_right,just_right_mentions, twitter_rank,friday_close,three_month_volume)



['USWS', 'NAKD', 'CCIV', 'TTOO', 'TMBR', 'SNDL', 'ZOM', 'CLOV', 'JUPW', 'LMFA', 'ACST', 'ISR', 'BNGO', 'AVGR', 'XSPA', 'GOVX', 'THMO', 'SIFY', 'TRXC', 'MESO', 'GEVO', 'CTRM', 'AIKI', 'DFFN', 'NSPR', 'RIOT', 'KMPH', 'NBY', 'CRBP', 'EXPC', 'TLRY', 'CIDM', 'RIDE', 'SLGG', 'NTEC', 'ACB', 'IDEX', 'OCGN', 'GHSI', 'BRQS', 'CHNR', 'ELTK', 'TGC', 'BW', 'AEZS', 'NPA', 'GRNQ', 'MGNI', 'MARA', 'NMRD', 'INUV', 'GSL', 'NOVN', 'ATHE', 'BORR', 'UPWK', 'GME', 'FCEL', 'NGA', 'SYN', 'SOS', 'ADXS', 'TYME', 'LPCN', 'PHIO', 'KERN', 'NBRV', 'DMTK', 'WTER', 'CANF', 'BFT', 'SALM', 'GIK', 'AWX', 'FUV', 'DDD', 'XONE', 'SUNW', 'PECK', 'CREG', 'FTFT', 'FSR', 'CRON', 'APHA', 'EQOS', 'ACAM', 'QLGN', 'CEMI', 'HUGE', 'PIXY', 'RAVE', 'ZYNE', 'CGC', 'HEXO', 'TTNP', 'USWSW', 'KRUS', 'STAF', 'SNMP', 'ONTX', 'REED', 'PED', 'BOX', 'DPW', 'ALRN', 'SHIP', 'EKSO', 'PSAC', 'VOXX'] [21, 21, 20, 13, 10, 9, 9, 8, 8, 7, 7, 6, 6, 6, 6, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,

In [23]:
#############################################################set export file path#######
workbook  = xlsxwriter.Workbook('/Users/jpjansen/Documents/1_TESTTEST_WeBullStats.xlsx')
############################################################################
worksheet = workbook.add_worksheet()
worksheet.write(0,0,'Stock Symbol')
worksheet.write(0,1,'Twitter Rank')
worksheet.write(0,2,'Mentions')
worksheet.write(0,3,'Friday Close')
worksheet.write(0,4, 'Friday Vol')
worksheet.write(0,5, '3 Month Vol')
worksheet.write(0,6, '52 Week Min')
worksheet.write(0,7, '52 Week Max')
for i in range(len(just_right)):
    worksheet.write(i+1, 0,just_right[i])
    worksheet.write(i+1, 1,twitter_rank[i])
    worksheet.write(i+1, 2,just_right_mentions[i])
    worksheet.write(i+1, 3,friday_close[i])
    worksheet.write(i+1, 4,friday_volume[i])
    worksheet.write(i+1, 5,three_month_volume[i])
    worksheet.write(i+1, 6,fifty_two_week_low[i])
    worksheet.write(i+1, 7,fifty_two_week_high[i])
workbook.close()